In [224]:
from os import walk
from os.path import join
import pandas as pd
import matplotlib.pyplot as plt
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer

from bs4 import BeautifulSoup
%matplotlib inline


In [225]:
Example_file = "C:/Users/Swami/Downloads/SpamData/SpamData/01_Processing/practice_email.txt"
Spam_1_path = "C:/Users/Swami/Downloads/SpamData/SpamData/01_Processing/spam_assassin_corpus/spam_1"
Spam_2_path = "C:/Users/Swami/Downloads/SpamData/SpamData/01_Processing/spam_assassin_corpus/spam_2"
Easy_Nonspam_1_path = "C:/Users/Swami/Downloads/SpamData/SpamData/01_Processing/spam_assassin_corpus/easy_ham_1"
Easy_Nonspam_2_path = "C:/Users/Swami/Downloads/SpamData/SpamData/01_Processing/spam_assassin_corpus/easy_ham_2"

DATA_JSON_FILE = "C:/Users/Swami/Downloads/SpamData/SpamData/01_Processing/email_text-data.json"
Spam_Cat = 1
Ham_Cat = 0

In [226]:
stream = open(Example_file, encoding='latin-1')
message = stream.read()
stream.close()

print(message)

From exmh-workers-admin@redhat.com  Thu Aug 22 12:36:23 2002
Return-Path: <exmh-workers-admin@spamassassin.taint.org>
Delivered-To: zzzz@localhost.netnoteinc.com
Received: from localhost (localhost [127.0.0.1])
	by phobos.labs.netnoteinc.com (Postfix) with ESMTP id D03E543C36
	for <zzzz@localhost>; Thu, 22 Aug 2002 07:36:16 -0400 (EDT)
Received: from phobos [127.0.0.1]
	by localhost with IMAP (fetchmail-5.9.0)
	for zzzz@localhost (single-drop); Thu, 22 Aug 2002 12:36:16 +0100 (IST)
Received: from listman.spamassassin.taint.org (listman.spamassassin.taint.org [66.187.233.211]) by
    dogma.slashnull.org (8.11.6/8.11.6) with ESMTP id g7MBYrZ04811 for
    <zzzz-exmh@spamassassin.taint.org>; Thu, 22 Aug 2002 12:34:53 +0100
Received: from listman.spamassassin.taint.org (localhost.localdomain [127.0.0.1]) by
    listman.redhat.com (Postfix) with ESMTP id 8386540858; Thu, 22 Aug 2002
    07:35:02 -0400 (EDT)
Delivered-To: exmh-workers@listman.spamassassin.taint.org
Received: from int-mx1.corp

Display mail body from the text file

In [209]:
stream = open(Example_file, encoding='latin-1')

is_body = False
lines = []
for line in stream:
 if is_body:
  lines.append(line)
 elif line =='\n': # if there is a blank line between paragraph, if it is new line then it is body of mail
   is_body = True
   
stream.close()
email_body = '\n' .join(lines) # new line charac makes to display form list like a paragraph
print(email_body)





Dear Mr Still



Good tidings to you and all your staff for the festive season ahead (Christmas).

Now to the crux of the matter-in-hand: I am a fully qualified Santa Claus and am wondering whether you might consider me to run my own "Santa's Grotto" in your store.

But WAIT! You're probably thinking: "What makes him so special?"

Well, first of all, I have made several changes to the characterisation of Father Christmas. Rather than greeting the children with shouts of "Ho, ho, ho!" I prefer to whisper the phrase "Dependence is not unfathomable in this cruel world we live in". In addition, my gifts are ALL hand-made, ranging from felt hoops to vanilla-pod holders.

You will note also, from the enclosed sketch, that I have radically redesigned Santa's outfit and have renamed my character "Lord Buckles". Would you be interested in employing me? I promise NEVER to let you down.

I look forward to hearing from you.



Best wishes

Robin Cooper

[Excerpt from the book: The Timewaster Let

Generator function

In [210]:
def generate_squares(N) :
    for my_number in range(N):
     yield my_number ** 2

In [211]:
for i in generate_squares(5):
    print(i, end='>')

0>1>4>9>16>

Email body extraction

In [230]:
def email_body_generator(path):
 for root, dirnames, filenames in walk(path):
     for file_name in filenames:
        filepath = join(root, file_name)

        stream = open(filepath, encoding='latin-1')

        is_body = False
        lines = []
        for line in stream:
           if is_body:
             lines.append(line)
           elif line =='\n': # if there is a blank line between paragraph, if it is new line then it is body of mail
            is_body = True
        
        stream.close()
        email_body = '\n' .join(lines) # new line charac makes to display form list like a paragraph
        yield file_name, email_body


In [231]:
def df_from_directory(path, classification):
    rows = []
    row_names = []
    
    for file_name, email_body in email_body_generator(path):
        rows.append({'MESSAGE': email_body, 'CATEGORY': classification})
        row_names.append(file_name)
        
return pd.DataFrame(rows, index=row_names)

In [232]:
spam_emails = df_from_directory(Spam_1_path, 1)
spam_emails = spam_emails.append(df_from_directory(Spam_2_path, 1))
spam_emails.head()

AttributeError: 'DataFrame' object has no attribute 'append'

In [ ]:
ham_emails = df_from_directory(EASY_NONSPAM_1_PATH, HAM_CAT)
ham_emails = ham_emails.append(df_from_directory(EASY_NONSPAM_2_PATH, HAM_CAT))
ham_emails.shape

In [229]:
data = pd.concat([spam_emails, ham_emails])
print('Shape of entire dataframe is', data.shape)
data.head()


NameError: name 'ham_emails' is not defined

Data Cleaning : Checking for Missing values

In [ ]:
# check if any message bodies are null
data['MESSAGE'].isnull().values.any()
 # we have no empty values in message

In [ ]:

# empty string will be like this " ", 
len ("")

In [ ]:
# Check if there are empty emails (string length zero).
(data.MESSAGE.str.len() == 0).any().sum() #( how many have zero strings)

In [ ]:
# how would you check the no. of entries with null/None values
data.MESSAGE.isnull().sum()

LOCATE Empty Mails

In [ ]:
data[data.MESSAGE.str.len() == 0].index

In [ ]:
 data.index.get_loc('cmds')

# Remove system file entries from dataframe

In [ ]:
data =data.drop(['cmds'], inplace = True)

ADD document ID to track Emails in Dataset

In [ ]:
document_ids = range(0, len(data.index))
data['DOC_ID'] = document_ids

In [ ]:
data['FILE_NAME'] = data.index
data.set_index('DOC_ID', inplace=True) # shows as seperate index
data.head()

Save to File using Pandas

In [ ]:
data.to_json(DATA_JSON_FILE)

Number of Spam Messages Visualization

In [ ]:
data.CATEGORY.value_counts() # displays 2 category spam and ham

In [ ]:
amount_of_spam = data.CATEGORY.value_counts()[1] # selecting spam and ham category
amount_of_ham = data.CATEGORY.value_counts()[0]

In [ ]:
category_names = ['Spam', 'Legit_Mail']
sizes = [amount_of_spam, amount_of_ham]
plt.figure(figsize=(2,2), dpi= 227) # density of pixels per inch
plt.pie(sizes, labels=category_names, textprops={'fontsize': 6}, 
startangle=10, auto = '%1.1f%%', explode=[0,0.1]) # startangle changes pie position, 
#1.1 shows one decimal point, explode means seprate the pie with one another.

# draw  circle
plt.Circle((0,0), radius=0.6, fc='white')
plt.gca().add_artist(center_circle)
plt.show()

# Natural Language Processing
Text processing ( Tokenizer & Stopwords)

In [ ]:
nltk.download('punkt') # it downloads some packages

In [ ]:
# stop words - at, of the ,a at, and dont meain word seperation for NLP
nltk.download('stopwords' ) # downloads
stop_words = set(stopwords.words('english'))
if 'this' in stop_words: print('Found it')

In [ ]:
msg = " All work and no play makes jack a dull boy."
word = word_tokenize(msg.lower())
filtered_words = []
# Append non-stop words to filtered words
for word in words:
    if word not in stop_words:
        filtered_words.append(word)
print(filtered_words)

 Word Stemming (like fishing, fished, fisher are came form FISH)

In [ ]:
msg = " All work and no play makes jack a dull boy. \
    Nobody Expect this! "
word = word_tokenize(msg.lower())

stemmer = PorterStemmer( )
stemmer = SnowballStemmer('Finnish') # to stemm word which is not english u can provide the language here
filtered_words = []
# Append non-stop words to filtered words
for word in words:
    if word not in stop_words:
        stemmed_word = stemmer.stem(word)
        filtered_words.append(stemmed_word) # here makes is stemmed to make.
print(filtered_words)

# Removing Punctuation

In [ ]:
msg = ' All work and no play makes jack a dull boy.???? \
    Nobody Expect this! '
word = word_tokenize(msg.lower())
stemmer = SnowballStemmer('Finnish') # to stemm word which is not english u can provide the language here
filtered_words = []
# Append non-stop words to filtered words
for word in words:
    if word not in stop_words and word.isalpha(): # isalpha identifies punctuation 
        stemmed_word = stemmer.stem(word)
        filtered_words.append(stemmed_word) # here makes is stemmed to make.
print(filtered_words)

Removing HTML tags from E-MAILS

In [ ]:
# data.at[2, 'MESSAGE'] # This is at 2nd index and MESSAGE Column
soup = BeautifulSoup(data.at[2, 'MESSAGE'], 'html.parser')
print(soup.prettify() ) 


In [ ]:
soup.get_text() # --> removes all html tags

Functions for Email Processing

In [ ]:
# Modifies func to remove HTML tags, then test on Email with DOC_ID 2
  def clean_msg_no_html( message, stemmer=PorterStemmer(), 
                     stop_words=set(stopwords.words('english'))):
         
         # Remove Html Tags
        soup = BeautifulSoup(message, 'html.parser')
        cleaned_text =  soup.get_text( ) 
        # Converts  to lower case and split the words
        words = word_tokenize(cleaned_text.lower())
        
        filtered_words = []

        for word in words:
            # Remove the stop words and punctuation
             if word not in stop_words and word.isalpha():
                filtered_word.append(stemmer.stem(word))


         return filtered_words

In [ ]:
clean_msg_no_html(data.at[2, 'MESSAGE']) #giving this input as 2nd 
#Doc_id and its message

 Applying Cleaning and tokenization to all messages

 Slicing Dataframes and Series & Creating Subsets

In [ ]:
 data.iat[2,  1] #2nd one is column number 1 is message column
 #, 0 is category column

In [ ]:
data.iloc[0:2] # get rows of dataframe

In [ ]:
first_emails = data.MESSAGE.iloc[0:3]

nested_list = first_emails.apply(clean_message)